# setup

In [6]:
%matplotlib inline
from eelbrain import *
import numpy as np
import pandas as pd
import scipy, os, pdb, importlib
import matplotlib.pyplot as plt
import seaborn as sns

# color definitions
colorC = (0.15,0.7,0.1)
colorPi = (0.7,0.15,0.7)
colorPc = (1, 0.5, 1)

colC = (0.15,0.7,0.1)
colP = (0.8,0.15,0.8)

root_folder = 'data_path'
output_folder = 'output_path'
plot_folder = f'{output_folder}/plots'
if not os.path.exists(plot_folder):
    os.makedirs(plot_folder)

meg_folder = f'{root_folder}/meg'

subjects = [f for f in os.listdir(meg_folder) if f[0]=='R']
subjects.sort()

CONTROLS = ['R2517', 'R2519', 'R2520', 'R2521', 'R2525', 'R2528', 'R2496', 'R2673',] 
PATIENTS = ['R2527', 'R2540', 'R2546', 'R2598', 'R2615', 'R2617', 'R2664', 'R2667', 'R2668',]

# Fig 1A brains

In [5]:
psdP = []
psdC = []
subjsP  = []
subjsC = []
for subject in subjects:
    if subject=='R26672':
        continue
    print(subject, end='\r')
    psds = []
    for visit in ['visit1', 'visit2']:
        for task in ['resting', 'visual_fam', 'visual_mm', 'visual_pn', 'visual_pd']:
            datafile = f'{output_folder}/{subject}_{visit}_{task}_psd.pkl'
            if os.path.exists(datafile):
                psd = load.unpickle(datafile)
                psds.append(psd.sub(frequency=(13, 25)).mean('frequency'))
            if subject=='R2667':
                datafile = f'{output_folder}/R26672_{visit}_{task}_psd.pkl'
                if os.path.exists(datafile):
                    psd = load.unpickle(datafile)
                    psds.append(psd.sub(frequency=(13, 25)).mean('frequency'))
    if subject in PATIENTS:
        psdP.append(combine(psds).mean('case'))
        subjsP.append(subject)
    else:
        psdC.append(combine(psds).mean('case'))
        subjsC.append(subject)

print('smoothing')
psdP = combine(psdP).smooth('source',0.005,'gaussian')
psdC = combine(psdC).smooth('source',0.005,'gaussian')

medial = ['lingual', 'medial', 'parahippocampal', 'cuneus', 'precuneus','cingulate','unknown']
laterals = list(set([l for l in psdP.source.parc.as_labels() if not np.any([v in l for v in medial])]))
rolandicROI = list(set([l for l in psdP.source.parc.as_labels() if 'central' in l]))

smoothing


In [7]:
psdPplot = psdP.copy()
psdCplot = psdC.copy()
psdPplot.x /= 1e-24 # convert units
psdCplot.x /= 1e-24

vmin = 0
vmax = 350

outfolder1 = f'{plot_folder}/brain_inferno_{vmax:.2f}_{vmin:.2f}'
 
if not os.path.exists(outfolder1):
    os.makedirs(outfolder1)

from matplotlib import cm
from matplotlib.colors import ListedColormap

cmap = cm.get_cmap('inferno', 24)
cmapvals = cmap(np.linspace(0,1,512))
cmapvals[:1,:] = [0,0,0,0] # make transparent colormap
cmapvals[1:,3] = np.linspace(0,1,511)
cmap = ListedColormap(cmapvals)

 
p = plot.brain.brain(psdPplot.sub(source=laterals).mean('case'), vmax=vmax, vmin=vmin, cmap=cmap, surf='pial', h=1000)
p.save_image(f'{outfolder1}/brainP_pial_{vmax:.2f}_{vmin:.2f}.png')
p.close()
p = plot.brain.brain(psdCplot.sub(source=laterals).mean('case'), vmax=vmax, vmin=vmin, cmap=cmap, surf='pial', h=1000)
p.save_image(f'{outfolder1}/brainC_pial_{vmax:.2f}_{vmin:.2f}.png')
p.close()

# Fig 1 B, D spectrum

### plot_beta_power

In [8]:
def plot_beta_power(Cs, Ps, outfolder, savestr=''):
    Cs.x /= 1e-24
    Ps.x /= 1e-24

    colC = (0.15,0.7,0.1)
    colP = (0.8,0.15,0.8)
  
    plt.figure(figsize=(15,10))
    Cmean = np.mean(Cs.x, axis=0)
    Cstd = np.std(Cs.x, axis=0)/np.sqrt(len(Cs))
    plt.plot(Cs.frequency.values, Cmean, color=colC, linewidth=5)
    plt.fill_between(Cs.frequency.values, Cmean-Cstd, Cmean+Cstd, color=colC, alpha=0.5)
    Cmean = np.mean(Ps.x, axis=0)
    Cstd = np.std(Ps.x, axis=0)/np.sqrt(len(Ps))
    plt.plot(Ps.frequency.values, Cmean, color=colP, linewidth=5)
    plt.fill_between(Ps.frequency.values, Cmean-Cstd, Cmean+Cstd, color=colP, alpha=0.5)
    plt.title('First Visit', fontsize=15)
    plt.xlabel('Frequency [Hz]', fontsize=15)
    plt.gca().tick_params(axis='x', labelsize=15)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.ylabel('Response Power', fontsize=15)
    plt.gca().spines['left'].set_linewidth(4)
    plt.gca().spines['bottom'].set_linewidth(4)
    plt.gca().spines['left'].set_linewidth(4)
    plt.gca().spines['bottom'].set_linewidth(4)
    plt.gca().tick_params(width=4, length=10)
    plt.gca().set_xticklabels([])
    plt.ylim([0, 400])
    plt.gca().set_yticks([0, 100, 200, 300, 400])
    plt.gca().set_yticklabels([])
    plt.ylabel('')
    plt.xlabel('')
    plt.title('')
    plt.savefig(f'{outfolder}/001_beta_spectrum_{savestr}.png', bbox_inches='tight')

In [ ]:
outfolder = f'{output_folder}/beta_power'
if not os.path.exists(outfolder):
    os.makedirs(outfolder)
    
psdP = dict(visit1=[],visit2=[])
psdC = dict(visit1=[],visit2=[])
subjsP  = dict(visit1=[],visit2=[])
subjsC = dict(visit1=[],visit2=[])
for subject in subjects:
    if subject=='R26672':
        continue
    print(subject, end='\r')
    for visit in ['visit1', 'visit2']:
        psds = []
        for task in ['resting', 'visual_fam', 'visual_mm', 'visual_pn', 'visual_pd']:
            datafile = f'{output_folder}/{subject}_{visit}_{task}_psd.pkl'
            if os.path.exists(datafile):
                psd = load.unpickle(datafile)
                psds.append(psd.sub(source=rolandicROI).mean('source').sub(frequency=(2,40)))
            if subject=='R2667':
                datafile = f'{output_folder}/R26672_{visit}_{task}_psd.pkl'
                if os.path.exists(datafile):
                    psd = load.unpickle(datafile)
                    psds.append(psd.sub(source=rolandicROI).mean('source').sub(frequency=(2,40)))
        if len(psds) > 0:
            if subject in PATIENTS:
                psdP[visit].append(combine(psds).mean('case'))
                subjsP[visit].append(subject)
            else:
                psdC[visit].append(combine(psds).mean('case'))
                subjsC[visit].append(subject)    
    
plot_beta_power(combine(psdC['visit1']), combine(psdP['visit1']), outfolder, savestr='visit1')
plot_beta_power(combine(psdC['visit2']), combine(psdP['visit2']), outfolder, savestr='visit2')


psdC2 = []
psdP2 = []
for subject in subjsC['visit2']:
    i = subjsC['visit1'].index(subject)
    psdC2.append(psdC['visit1'][i])
for subject in subjsP['visit2']:
    i = subjsP['visit1'].index(subject)
    psdP2.append(psdP['visit1'][i])

plot_beta_power(combine(psdC2), combine(psdP2), outfolder, savestr='visit2inset')

# Fig 1 C, E boxplots

### plot_lesion_box

In [10]:
def plot_lesion_box(df, tstr, plotfolder, ylim=None, yticks=None, plottype='box'):
    dfC = df[df['group']=='C']
    dfP = df[df['group']=='P']
    boxC = np.asarray(dfC.groupby('subject', as_index=False)['y'].mean()['y'])
    dfPi = dfP[dfP['lesion']=='ipsi']
    boxPi = np.asarray(dfPi.groupby('subject', as_index=False)['y'].mean()['y'])
    dfPc = dfP[dfP['lesion']=='contra']
    boxPc = np.asarray(dfPc.groupby('subject', as_index=False)['y'].mean()['y'])
    print(len(boxC), len(boxPi), len(boxPc))

    plt.figure(figsize=(5,10))
    if plottype == 'box':
        boxax = plt.boxplot([boxC, boxPc, boxPi], positions=[1,2,3], patch_artist=True,
                       medianprops=dict(color='k', linewidth=2), boxprops=dict(linewidth=2), whiskerprops=dict(linewidth=2))
        colors = [colorC, colorPc, colorPi]
        for patch, color in zip(boxax['boxes'], colors):
            patch.set_facecolor(color)
    elif plottype == 'scatter':
        plt.scatter([0]*len(boxC), boxC, color=colorC)
        plt.scatter([1]*len(boxPc), boxPc, color=colorPc)
        plt.scatter([2]*len(boxPi), boxPi, color=colorPi)
    plt.gca().set_xticklabels(['C','Pc','Pi'])
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['left'].set_linewidth(2)
    plt.gca().spines['bottom'].set_linewidth(2)
#     plt.title(tstr)
    plt.gca().set_yticks(yticks)
    plt.gca().set_yticklabels([])
    plt.gca().set_xticklabels([])
    plt.gca().tick_params(axis='y',width=2, length=10)
    plt.gca().tick_params(axis='x',width=2, length=10)
    if ylim is not None:
        plt.ylim(ylim)
    plt.savefig(f'{plotfolder}/boxplot_{tstr}_{plottype}.png', bbox_inches='tight')

In [ ]:
plotfolder = f'{plot_folder}/boxplots'
if not os.path.exists(plotfolder):
    os.makedirs(plotfolder)

# df = pd.read_csv('betapower_lesionhemi_for_plots.csv')
df = pd.read_csv('betapower_lesionhemi_for_plots.csv')
df['y'] = 100*df['rel_beta']

plot_lesion_box(df[df['visit']=='visit1'], 'visit1', plotfolder, ylim=[20,70], yticks=[20, 30, 40, 50, 60, 70])
plot_lesion_box(df[df['visit']=='visit2'], 'visit2', plotfolder, ylim=[20,70], yticks=[20, 30, 40, 50, 60, 70])

# Fig 2 A, B, C, D spectrograms

### plot_ERDERS_Array

In [12]:
def plot_ERDERS_Array(ev, vmin, vmax, outfolder, savestr, newcmp='seismic', flipsign=False):
    if flipsign:
        p = plot.Array(-ev, vmin=vmin, vmax=vmax, w=10, h=5)
    else:
        p = plot.Array(ev, vmin=vmin, vmax=vmax, w=10, h=5)
    i=0
    for ax in p.figure.axes:
        for aa in ['top', 'right']:
            ax.spines[aa].set_visible(False)
        for aa in ['bottom', 'left']:
            ax.spines[aa].set_lw(1.5)
        ax.xaxis.set_tick_params(width=1.5, length=7)
        ax.yaxis.set_tick_params(width=1.5, length=7)
        ax.title.set_text(savestr)
        ax.title.set_fontsize(15)
        ax.xaxis.set_ticklabels([-3, -2, -1, 0, 1, 2, 3], fontsize=12)
        ax.spines['bottom'].set_linewidth(4)
        ax.spines['left'].set_linewidth(4)
        ax.tick_params(width=4, length=20)
        if i > 1:
            ax.set_xlabel('Time [s]', fontsize=12)
        if i==0 or i==2:
            ax.set_ylabel('Frequency [Hz]', fontsize=12)
        ax.axhline(ax13, linewidth=2, linestyle='dashed', color='k')
        ax.axhline(ax25, linewidth=2, linestyle='dashed', color='k')
        i+=1 
    p.save(f'{outfolder}/{savestr}.png')

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap

cmap = cm.get_cmap('xpolar', 256)

CONTROLS = [ 'R2517', 'R2519', 'R2520', 'R2521', 'R2525', 'R2528', 'R2496', 'R2673',]
PATIENTS = [ 'R2527', 'R2540', 'R2546', 'R2598', 'R2615', 'R2617', 'R2664', 'R2667', 'R2668',]
subjects = CONTROLS+PATIENTS

spec_dict = {}
for visit in ['visit1', 'visit2']:
    specs_all = []
    for subject in CONTROLS:
        specs = []
        for task in ['mm', 'pn', 'pd']:
            infile = f'{subject}_{visit}_visual_{task}_button_specgramsL.pkl'
            if not os.path.exists(f'{output_folder}/{infile}'):
                print(f'file not found: {infile}')
                continue
            print(f'loading: {infile}')
            specL = load.unpickle(f'{output_folder}/{infile}')
            specR = load.unpickle(f'{output_folder}/{subject}_{visit}_visual_{task}_button_specgramsR.pkl')
            spec = combine(specL + specR).mean('case')
            normf = spec.sub(time=(-2.9,-2)).mean('time').sub(frequency=(13,25)).mean('frequency')
            spec = (spec -  spec.sub(time=(-2.9,-2)).mean('time')) / normf
            specs.append(spec.sub(time=(-2.9, 2.9)))
        if len(specs) > 0:
            specs_all.append(combine(specs).mean('case'))
    spec_dict['C'+visit] = combine(specs_all).copy()
    
    specs_all = []
    for subject in PATIENTS:
        specs = []
        for task in ['mm', 'pn', 'pd']:
            infile = f'{subject}_{visit}_visual_{task}_button_specgramsL.pkl'
            if not os.path.exists(f'{output_folder}/{infile}'):
                print(f'file not found: {infile}')
                continue
            print(f'loading: {infile}')
            specL = load.unpickle(f'{output_folder}/{infile}')
            specR = load.unpickle(f'{output_folder}/{subject}_{visit}_visual_{task}_button_specgramsR.pkl')
            spec = combine(specL + specR).mean('case')
            normf = spec.sub(time=(-2.9,-2)).mean('time').sub(frequency=(13,25)).mean('frequency')
            spec = (spec -  spec.sub(time=(-2.9,-2)).mean('time')) / normf
            specs.append(spec.sub(time=(-2.9, 2.9)))
        if len(specs) > 0:
            specs_all.append(combine(specs).mean('case'))
    spec_dict['P'+visit] = combine(specs_all)

vmax = np.max([spec_dict[k].mean('case').abs().max() for k in spec_dict.keys()])
vmin = -vmax
print(vmax, vmin)

for k in spec_dict.keys():
    plot_ERDERS_Array(spec_dict[k].mean('case'), vmin, vmax, plot_folder, k, newcmp=cmap)

# Fig 2 E F timecourse

In [ ]:
ev_dict = {}
for k in spec_dict.keys():
    ev_dict[k] = spec_dict[k].sub(frequency=(13,25)).mean('frequency')

vmax = np.max([ev_dict[k].abs().max() for k in ev_dict.keys()])
vmin = -vmax
print(vmax, vmin)

plt.figure(figsize=(10,5))
plt.plot(ev_dict['Cvisit1'].time.times, ev_dict['Cvisit1'].x.T, color=colC,alpha=0.8, linewidth=4)
plt.plot(ev_dict['Pvisit1'].time.times, ev_dict['Pvisit1'].x.T, color=colP,alpha=0.8, linewidth=4)
plt.ylim([1.1*vmin, 1.1*vmax])
plt.axhline(0, color='black', linewidth=4)
plt.axvline(0, color='black', linestyle='dashed', linewidth=4)
plt.gca().set_yticklabels([])
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().xaxis.set_tick_params(labelsize=15)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['left'].set_linewidth(4)
plt.savefig(f'{plot_folder}/buttontime_visit1.png', bbox_inches='tight')

plt.figure(figsize=(10,5))
plt.plot(ev_dict['Cvisit2'].time.times, ev_dict['Cvisit2'].x.T, color=colC,alpha=0.8, linewidth=4)
plt.plot(ev_dict['Pvisit2'].time.times, ev_dict['Pvisit2'].x.T, color=colP,alpha=0.8, linewidth=4)
plt.ylim([1.1*vmin, 1.1*vmax])
plt.axhline(0, color='black', linewidth=4)
plt.axvline(0, color='black', linestyle='dashed', linewidth=4)
plt.gca().set_yticklabels([])
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().xaxis.set_tick_params(labelsize=15)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['left'].set_linewidth(4)
plt.savefig(f'{plot_folder}/buttontime_visit2.png', bbox_inches='tight')

# Fig 2 G H boxplots

In [ ]:
plotfolder = f'{plot_folder}/boxplots'
if not os.path.exists(plotfolder):
    os.makedirs(plotfolder)

df = pd.read_csv('ERD_ERS_lesionhemi_for_plots.csv')

df['y'] = 100*df['value']

df_ERS = df[df['metric']=='ERS']
df_ERD = df[df['metric']=='ERD']

ylim = [0, 250]
yticks = [0,50,100,150,200,250]

plot_lesion_box(df_ERS[df_ERS['visit']=='visit1'], 'ERS_visit1', plotfolder, ylim=ylim, yticks=yticks)
plot_lesion_box(df_ERS[df_ERS['visit']=='visit2'], 'ERS_visit2', plotfolder, ylim=ylim, yticks=yticks)
plot_lesion_box(df_ERD[df_ERD['visit']=='visit1'], 'ERD_visit1', plotfolder, ylim=ylim, yticks=yticks)
plot_lesion_box(df_ERD[df_ERD['visit']=='visit2'], 'ERD_visit2', plotfolder, ylim=ylim, yticks=yticks)